In [1]:
import sys
sys.path.append('../')

# Terms, Formulas and Interpretations

Now we have all the elements to formally define ```Tarski``` languages:

**Definition** (Many-Sorted First-Order Language). A _many-sorted_ _first-order_ language ${\cal L}$ is made up of:
 - A non-empty set $T$ of _sorts_
 - An _infinite number_ of _variables_ $x_{1}^{\tau}, x_{2}^{\tau}, \ldots$ for each short $\tau \in T$
 - For each $n \geq 0$ and each tuple $(\tau_1, \ldots, \tau_{n+1}) \in T^{n+1}$ of sorts, a (possibly empty) set of _function_ symbols, each of which is said to have _arity_ and _type_ $(\tau_1, \ldots, \tau_{n+1})$
 - For each $n \geq 0$ and each tuple $(\tau_1, \ldots, \tau_{n+1}) \in T^{n}$ of sorts, a (possibly empty) set of _relation_ symbols (predicates), each of which is said to have _arity_ and _type_ $(\tau_1, \ldots, \tau_{n})$

Continuing with our ```Blocks World``` themed example

In [2]:
import tarski

# 1. Create language used to describe world states and transitions
bw = tarski.language()

# 2. Define sorts
place = bw.sort('place')
block = bw.sort('block', [place])

# 3. Define functions
loc = bw.function( 'loc', block, place )

# 4. Define predicates
clear = bw.predicate( 'clear', block)

_Constants_ are 0-arity functions, whose sort $\tau$ is a set with one single element. Hence, we handle them separatedly, as we specialise their representation

In [3]:
# 5. Define constants
b1, b2, b3, b4 = bw.const(('b_{}'.format(k) for k in (1,2,3,4)), block)
table = bw.const('table', place)

## (First-Order) Terms

Combinations of variables, functions and constants are called _terms_, and the rules for constructing them are given inductively:

**Definition** (First-Order Terms). A term $t$ can be:

 - Any variable $x^{\tau}$ of the language can be a term $t$ with type $\tau$
 - Any constant symbol of the language with type $\tau$ is a term with the same type
 - If $t_1, \ldots, t_n$ are terms with respective types $\tau_1, \ldots, \tau_n$ and $f$ is a _function_ symbol with type $(\tau_1, \ldots, \tau_n, \tau{n+1})$ then $f(t_1,\ldots,t_n)$ is a term with type $\tau_{n+1}$.

Terms are implemented as Python objects. Every constant symbol is an instance of ```Term```

In [4]:
from tarski import Term

isinstance(b1,Term)

True

Function symbols allow to nest terms, thus 

In [11]:
t1 = loc(b1)
isinstance(t,Term)

True

In [12]:
x = bw.var('x', block)
t2 = loc(x)
isinstance(t2,Term)

True

In [14]:
t3 = loc( loc(x) )
isinstance(t3,Term)

True

are all terms. ```Tarski``` textual representation of variables is a bit different

In [15]:
print(t1)
print(t2)
print(t3)

loc(b_1)
loc(?x)
loc(loc(?x))


in order to make distinct variables from constants, the former are printed with the prefix ```?```. 

### Built-in Function Symbols

## Formulas

## Interpretations

The semantics of a first--order language are formalised as the notion of _interpretation_, which is also typically referred in texts on computational logic as _model_ or _structure_. Informally, the interpretation of a first-order language is a "table" that allows to:

 1. Map a first--order _term_ to a constant symbol